In [11]:
from pyspark.sql import SparkSession, Row
from fill_f101_round_f import fill_f101_round_f
from datetime import datetime
import time
import uuid

spark = SparkSession.builder \
    .appName("fill_f101_round_f") \
    .enableHiveSupport() \
    .getOrCreate()

In [12]:
df = fill_f101_round_f('2018-02-01', spark)
start_time = datetime.now()
job_name = "fill_f101_round_f"
status = "SUCCESS"
df.write \
      .mode("overwrite") \
      .partitionBy("ledger_account") \
      .format("parquet") \
      .saveAsTable("DM.DM_F101_ROUND_F")
time.sleep(5)

end_time = datetime.now()

log_row = Row(
    id=str(uuid.uuid4()),
    job_name=job_name,
    start_time=start_time,
    end_time=end_time,
    status=status
)
log_df = spark.createDataFrame([log_row])
log_df.write.mode("append").insertInto("logs.LOGS")